In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import os
import joblib

In [ ]:
# Load the RandomForestClassifier model
model_filename = "hand_gesture_model.pkl"
try:
    with open(model_filename, 'rb') as f:
        model = joblib.load(f)  # Load the trained model using joblib
except OSError as e:
    print("Error:", e)  # Print the error message if the file cannot be opened
    print("File:", model_filename)  # Print the filename causing the error
    print("File exists:", os.path.exists(model_filename))  # Check if the file exists
    exit()  # Exit the program if an error occurs

In [ ]:
cap = cv2.VideoCapture(0)  # Open the default camera
detector = HandDetector(maxHands=2)  # Initialize HandDetector for hand tracking
imgSize = 300  # Define the size for displaying images
offset = 20  # Define the offset for cropping hand region
font = cv2.FONT_HERSHEY_SIMPLEX  # Define the font for text rendering
fontScale = 1  # Define the font scale for text rendering
fontColor = (255, 255, 255)  # Define the font color for text rendering
lineType = 2  # Define the line type for text rendering

In [ ]:
while True:  # Main loop for processing each frame
    success, img = cap.read()  # Read a frame from the video capture
    
    hands, img = detector.findHands(img)  # Detect hands in the frame using HandDetector
    
    if hands:  # If hands are detected
        hand = hands[0]  # Get the first detected hand
        x, y, w, h = hand['bbox']  # Extract bounding box coordinates of the hand
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]  # Crop the region around the hand

        if imgCrop.size != 0:  # Check if the cropped image is not empty
            try:
                # Resize the cropped image to match the input size expected by the model
                imgResize = cv2.resize(imgCrop, (150, 150))  # Assuming the model expects 150x150 images

                # Flatten the resized image to create a 1D array
                input_data = imgResize.flatten().reshape(1, -1)

                # Make prediction using the RandomForestClassifier model
                prediction = model.predict(input_data)[0]

                # Display the predicted action label near the hand region
                cv2.putText(img, prediction, (x, y - 10), font, fontScale, fontColor, lineType)
            except cv2.error as e:  # Handle OpenCV errors
                print(f"OpenCV Error: {e}")
            except Exception as e:  # Handle other exceptions
                print(f"Error: {e}")

In [ ]:
 # Display the output image
    cv2.imshow("Image", img)

In [ ]:
 # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
